In [1]:
import os
import sys
import folium
import numpy as np
import pandas as pd
import pyproj as pj
import math
from folium.plugins import HeatMap
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import shift
import scipy.stats as stats
from scipy.stats import norm
import matplotlib.mlab as mlab
import pylab as pl
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
# gps.csv contains five features: user_id, record_time, app_name, fg_time_ms, interval_ms
app_usage_df = pd.read_csv("app_usage.csv")

In [22]:
app_count = app_usage_df.groupby(['app_name']).size().reset_index(name='count')

In [28]:
app_usage_df.sort_values(by='fg_time_ms', ascending=False)

,user_id,record_time,app_name,fg_time_ms,interval_ms
193020,1361,2017-02-12 09:25:24.723000,com.sonyericsson.home,132129777,132129777
204501,513,2017-02-26 00:00:02.799000,com.htc.powersavinglauncher,86399999,86400000
15809,1036,2017-02-12 15:13:51.561000,com.sec.android.app.camera,86399999,86400000
177608,513,2017-03-07 00:02:54.090000,com.htc.powersavinglauncher,86399999,86400000
177289,513,2017-03-04 00:36:48.448000,com.htc.powersavinglauncher,86399999,86400000
177169,513,2017-03-02 00:03:39.557000,com.htc.powersavinglauncher,86399999,86400000
177402,513,2017-03-05 00:08:05.444000,com.htc.powersavinglauncher,86399999,86400000
177041,513,2017-03-01 00:05:00.216000,com.htc.powersavinglauncher,86399999,86400000
204735,513,2017-02-28 00:00:25.893000,com.htc.powersavinglauncher,86399999,86400000
177500,513,2017-03-06 00:00:20.042000,com.htc.powersavinglauncher,86399999,86400000


In [4]:
classify_df = pd.read_csv("data_for_classify.csv")

In [5]:
classify_df.head()

,user_id,home_charge_perc,uncharge_perc,mode1_perc,mode2_perc,charge_count,charge_length,charge_cell_counts
0,514,0.681629,0.642100,0.356244,0.001656,83,27.048193,33
1,534,0.644766,0.671904,0.168615,0.159481,100,18.772277,13
2,551,0.899927,0.570759,0.422993,0.006248,56,49.175439,17
3,555,0.456202,0.721056,0.278944,0.000000,57,30.842105,17
4,559,0.970503,0.629722,0.370278,0.000000,36,58.444444,2


In [11]:
def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

In [12]:
diff(classify_df.user_id.unique().tolist(), app_usage_df.user_id.unique().tolist())

[1288, 1308, 1323]

In [13]:
valid_app_usage_df = app_usage_df[app_usage_df['user_id'].isin(classify_df.user_id.unique().tolist())]

In [17]:
valid_app_usage_df.groupby(['user_id']).size().reset_index(name='num_app_records').describe()

,user_id,num_app_records
count,38.000000,38.000000
mean,1067.421053,2381.973684
std,268.422985,2197.562648
min,514.000000,76.000000
25%,948.250000,232.000000
50%,1050.500000,2343.500000
75%,1311.750000,3818.500000
max,1364.000000,7701.000000
